##### Load libs:

In [1]:
import pandas as pd
import sqlite3

##### 1.Create connect:

In [2]:
connect = sqlite3.connect('../data/checking-logs.sqlite')

##### 2.Query:

Create DataFrame "test_results":

In [9]:
test_query = """
WITH user_first_view AS (
    SELECT 
        uid,
        MIN(first_view_ts) as first_view_time
    FROM test
    GROUP BY uid
),
lab_deltas AS (
    SELECT 
        t.uid,
        t.first_commit_ts,
        d.deadlines,
        (julianday(t.first_commit_ts) - julianday(datetime(d.deadlines, 'unixepoch'))) * 24 as diff_hours,
        uv.first_view_time,
        CASE 
            WHEN t.first_commit_ts < uv.first_view_time THEN 'до'
            ELSE 'после'
        END as time_period
    FROM test t
    JOIN deadlines d ON t.labname = d.labs
    JOIN user_first_view uv ON t.uid = uv.uid
    WHERE t.labname != 'project1'
)
SELECT 
    time_period as time,
    AVG(diff_hours) as avg_diff
FROM lab_deltas
GROUP BY time_period
ORDER BY time_period
"""

test_results = pd.read_sql(test_query, connect)
print(test_results)

    time    avg_diff
0     до  -61.156438
1  после -103.953310


Create DataFrame "control_results":

In [23]:
control_query = """
WITH user_first_view AS (
    SELECT 
        uid,
        MIN(first_view_ts) as first_view_time
    FROM control
    GROUP BY uid
),
lab_deltas AS (
    SELECT 
        t.uid,
        t.first_commit_ts,
        d.deadlines,
        (julianday(t.first_commit_ts) - julianday(datetime(d.deadlines, 'unixepoch'))) * 24 as diff_hours,
        uv.first_view_time,
        CASE 
            WHEN t.first_commit_ts < uv.first_view_time THEN 'до'
            ELSE 'после'
        END as time_period
    FROM control t
    JOIN deadlines d ON t.labname = d.labs
    JOIN user_first_view uv ON t.uid = uv.uid
    WHERE t.labname != 'project1'
)
SELECT 
    time_period as time,
    AVG(diff_hours) as avg_diff
FROM lab_deltas
GROUP BY time_period
ORDER BY time_period
"""

control_results = pd.read_sql(control_query, connect)
print(control_results)

    time    avg_diff
0     до  -99.901295
1  после -113.232196


##### 3.Close connect:

In [24]:
connect.close()

##### 4.Question

Вопрос: влияют ли посещения пользователем на дедлайн?<br>Ответ: ДА